In [ ]:
from numbers import Number
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
import os
import zipfile
from google.colab import files
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pandas as pd
import json
import random
from PIL import Image
from collections import Counter
from google.colab import drive


A continuación tenemos el proceso de selección de las categorías para el proceso de modelamiento, y la reasignación de las etiquetas sin perder el conocimiento de las etiquetas originales.

In [ ]:
a = [51,88,77,76,73,46,81,17,41,89,74,94,58,43,60,95,82,83,80,23,75,78,18,50] # categorías del conjunto de validación con al menos 10 imagenes
b = [73,77,51,46,82,43,94,89,58,30,90,74,83,41,95,78,53,81,75,88,80,57,92,72,44] # categorías del conjunto de validación con al menos 10 imagens
c = np.intersect1d(a, b).tolist()

ordered_to_id = {i: j for i, j in enumerate(c)}
id_to_ordered = {j: i for i, j in ordered_to_id.items()}
def convert_id(value, via):
  if via == 'to_id':
    return ordered_to_id[value]
  if via == 'to_ordered':
    return id_to_ordered[value]

Preprocesamiento de los datos para poder aplicar los experimientos

In [ ]:
Y_train_1 = tf.keras.utils.to_categorical(Y_train_correct)
Y_val_1 = tf.keras.utils.to_categorical(Y_valid_correct)
Y_test_1 = tf.keras.utils.to_categorical(Y_test_correct)

train_datagen_1 = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='constant'
)
train_gen_1 = train_datagen_1.flow(x=X_train,
                               y=Y_train_1,
                               batch_size=32)

valid_datagen_1 = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)
valid_gen_1 = valid_datagen_1.flow(x=X_valid,
                               y=Y_val_1,
                               batch_size=32)

Primer experimento, el cual se define por la capa de entrada para imágenes 224x224, seis bloques de capas convolucionales (una capa convolucional y una capa de pooling), capa de vectorización de las matrices de las capas convolucionales, dos capas densas de 128 neuronas cada una, un dropout y la capa de salida para la clasificación de 18 categorías.

In [ ]:
#Definimos el modelo en keras
conv_net = tf.keras.models.Sequential()

#Capa de entrada
width  =X_train.shape[1]
height  =X_train.shape[2]
conv_net.add(tf.keras.layers.Input(shape=(width, height, 3)))

#Primer bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 8, kernel_size = 5, activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

#Segundo bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 16,
    kernel_size = 4,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Tercer bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 32,
    kernel_size = 3,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Cuarto bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 64,
    kernel_size = 3,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Quinto bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 128,
    kernel_size = 3,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

#Sexto bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 256,
    kernel_size = 2,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

#capa flatten, que transforma cualquier arreglo multidimensional en un vector unidimensional.
conv_net.add(tf.keras.layers.Flatten())

#####Clasificador
# Capa densa intermedia
conv_net.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Capa densa intermedia
conv_net.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Agregamos dropout para regularización
conv_net.add(tf.keras.layers.Dropout(0.2))

# Capa de salida
conv_net.add(tf.keras.layers.Dense(units=18, #Número de categorías
                                   activation='softmax')) #Clasificación multiclase
conv_net.summary()
tf.keras.utils.plot_model(conv_net,show_shapes=True)

In [ ]:
conv_net.compile(loss='categorical_crossentropy',
                 optimizer=tf.optimizers.Adam(learning_rate=1e-5),
                 metrics = ["accuracy"])

checkpoint = tf.keras.callbacks.ModelCheckpoint(
                  filepath='best_weights_128N.h5', #################################################
                  monitor="val_loss",
                  mode="min",
                  save_best_only=True,
                  save_weights_only=True
              )

stopping = tf.keras.callbacks.EarlyStopping(
                monitor="val_loss",
                patience=10,
                mode="min",
                restore_best_weights=True
            )


total_samples = X_train.shape[0]
validation_samples= X_valid.shape[0]
batch_size = 32

# Entrenamos el modelo
history = conv_net.fit(train_gen_1,
                       steps_per_epoch=total_samples // batch_size,
                       epochs=50,
                       validation_data= valid_gen_1 ,
                       #validation_steps=validation_samples // batch_size,
                       #verbose=1,
                       callbacks=[checkpoint, stopping] )

Segundo experimento, el cual se define por la capa de entrada para imágenes 224x224, cinco bloques de capas convolucionales (una capa convolucional y una capa de pooling), capa de vectorización de las matrices de las capas convolucionales, dos capas densas de 256 neuronas cada una, un dropout y la capa de salida para la clasificación de 18 categorías.

In [ ]:
#Definimos el modelo en keras
conv_net = tf.keras.models.Sequential()

#Capa de entrada
width  =X_train.shape[1]
height  =X_train.shape[2]
conv_net.add(tf.keras.layers.Input(shape=(width, height, 3)))

#Primer bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 8, kernel_size = 5, activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

#Segundo bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 16,
    kernel_size = 4,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Tercer bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 32,
    kernel_size = 3,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Cuarto bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 64,
    kernel_size = 3,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Quinto bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 128,
    kernel_size = 3,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

#capa flatten, que transforma cualquier arreglo multidimensional en un vector unidimensional.
conv_net.add(tf.keras.layers.Flatten())

#####Clasificador
# Capa densa intermedia
conv_net.add(tf.keras.layers.Dense(units=256, activation='relu'))

# Capa densa intermedia
conv_net.add(tf.keras.layers.Dense(units=256, activation='relu'))

# Agregamos dropout para regularización
conv_net.add(tf.keras.layers.Dropout(0.2))

# Capa de salida
conv_net.add(tf.keras.layers.Dense(units=18, #Número de categorías
                                   activation='softmax')) #Clasificación multiclase

conv_net.summary()
tf.keras.utils.plot_model(conv_net,show_shapes=True)

In [ ]:
conv_net.compile(loss='categorical_crossentropy',
                 optimizer=tf.optimizers.Adam(learning_rate=1e-5),
                 metrics = ["accuracy"])

checkpoint = tf.keras.callbacks.ModelCheckpoint(
                  filepath='best_weights_256N.h5',
                  monitor="val_loss",
                  mode="min",
                  save_best_only=True,
                  save_weights_only=True
              )

stopping = tf.keras.callbacks.EarlyStopping(
                monitor="val_loss",
                patience=10,
                mode="min",
                restore_best_weights=True
            )


total_samples = X_train.shape[0]
validation_samples= X_valid.shape[0]
batch_size = 32

# Entrenamos el modelo
history = conv_net.fit(train_gen_1,
                       steps_per_epoch=total_samples // batch_size,
                       epochs=50,
                       validation_data= valid_gen_1 ,
                       #validation_steps=validation_samples // batch_size,
                       #verbose=1,
                       callbacks=[checkpoint, stopping] )

Tercer experimento, el cual se define por la capa de entrada para imágenes 224x224, cuatro bloques de capas convolucionales (una capa convolucional y una capa de pooling), capa de vectorización de las matrices de las capas convolucionales, dos capas densas de 512 neuronas cada una, un dropout y la capa de salida para la clasificación de 18 categorías.

In [ ]:
#Definimos el modelo en keras
conv_net = tf.keras.models.Sequential()

#Capa de entrada
width  =X_train.shape[1]
height  =X_train.shape[2]
conv_net.add(tf.keras.layers.Input(shape=(width, height, 3)))

#Primer bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 8, kernel_size = 6, activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

#Segundo bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 16,
    kernel_size = 5,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Tercer bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 32,
    kernel_size = 4,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Cuarto bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 64,
    kernel_size = 3,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))


#capa flatten, que transforma cualquier arreglo multidimensional en un vector unidimensional.
conv_net.add(tf.keras.layers.Flatten())

#####Clasificador
# Capa densa intermedia
conv_net.add(tf.keras.layers.Dense(units=512, activation='relu'))

# Capa densa intermedia
conv_net.add(tf.keras.layers.Dense(units=512, activation='relu'))

# Agregamos dropout para regularización
conv_net.add(tf.keras.layers.Dropout(0.2))

# Capa de salida
conv_net.add(tf.keras.layers.Dense(units=18, #Número de categorías
                                   activation='softmax')) #Clasificación multiclase

conv_net.summary()
tf.keras.utils.plot_model(conv_net,show_shapes=True)

In [ ]:
conv_net.compile(loss='categorical_crossentropy',
                 optimizer=tf.optimizers.Adam(learning_rate=1e-5),
                 metrics = ["accuracy"])

checkpoint = tf.keras.callbacks.ModelCheckpoint(
                  filepath='best_weights_512N.h5',
                  monitor="val_loss",
                  mode="min",
                  save_best_only=True,
                  save_weights_only=True
              )

stopping = tf.keras.callbacks.EarlyStopping(
                monitor="val_loss",
                patience=10,
                mode="min",
                restore_best_weights=True
            )


total_samples = X_train.shape[0]
validation_samples= X_valid.shape[0]
batch_size = 32

# Entrenamos el modelo
history = conv_net.fit(train_gen_1,
                       steps_per_epoch=total_samples // batch_size,
                       epochs=50,
                       validation_data= valid_gen_1 ,
                       #validation_steps=validation_samples // batch_size,
                       #verbose=1,
                       callbacks=[checkpoint, stopping] )